# Blackjack Strategies and Applications
### By: Sam Lyon

### 1. Abstract

Blackjack is a widely popular and ostensibly simple card game that appears at nearly every American casino. Despite its perceived simplicity, there are a number of strategies one can take to improve one's odds against the dealer, who, by design, has the natural advantage.

Given a list of assumptions that I outline in the "Background" section below, I explore some of these strategies throughtout this project. As one might expect, the more advanced the strategy, the more likely it is to beat the dealer. Here I will define "advanced" as: the player incorporates additional facets of the game beyond "hit" and "stay" and/or the player strategically bases each decision on the specific cards he holds as well as the one card the dealer has showing.

This project assumes a basic understanding of the rules of blackjack. This site will be helpful to fill in any gaps:
https://bicyclecards.com/how-to-play/blackjack/#:~:text=If%20a%20player's%20first%20two,the%20amount%20of%20their%20bet.

### 2. Background

Here are some assumptions I used for the project.

1. The game uses one deck of cards, which is reshuffled after each game iteration.
2. Cards are taken from this deck without replacement until the deck is reshuffled.
3. The dealer abides by the usual rules.
4. The player uses various strategies which I will outline as they come up.

I used a list of 52 integers to represent the deck, with each integer corresponding to one of the 52 cards in a standard deck. I utilized functions from the random package to randomly select cards as necessary.

I'll explain some of the necessary component functions and then proceed to the strategies.

### 3. Main Findings

In [1]:
import random

In [2]:
def hit(cards):
    new_card = random.choice(l_new)
    cards.append(new_card)
    l_new.remove(new_card)
    return cards

The **hit** function takes as input a list of cards corresponding to the dealer or player's hand. It selects at random a card from the deck, adds it to the hand, and then removes that card from the deck.

In [3]:
def ace_change(cards):
    if 11 in cards:
        cards.remove(11)
        cards.append(1)
        return cards
    else:
        return cards

The **ace_change** function takes a hand of cards, removes an 11 if there is one, and replaces the 11 with a 1. If there is no 11, the hand is returned untouched.

In [4]:
def compare(pc, dc, b):
    if isinstance(pc, list):
        return compare(pc[0], dc, b * 2)
    elif pc == "Bust":
        return -b
    elif dc == "BJ":
        if dc == pc:
            return 0
        else:
            return -b
    elif pc == "BJ":
        if dc != pc:
            return 1.5 * b
        else:
            return 0
    elif dc == "Bust":
        return b
    elif pc == dc:
        return 0
    elif pc > dc:
        return b
    else:
        return -b

The **compare** function takes as input the following variables. 

1. pc: The integer value of the sum of the player's cards. Could also be valued as "Bust" (if cards' sum over 21) or "BJ" (blackjack)

2. dc: The integer value of the sum of the dealer's cards. Could also be valued as "Bust" (if cards' sum over 21) or "BJ" (blackjack)

3. b: The amount the player bet on the hand.

The **compare** function returns the amount of money the player gained or lost on the hand.

In [5]:
def double(cards):
    hc = hit(cards)
    spc = sum(hc)
    if spc > 21:
        acc = sum(ace_change(hc))
        if acc > 21:
            spc = "Bust"
    return compare([spc], d_hand, bet)

The **double** function hits a hand once and then sends the sum of the cards to the **compare** function. If the sum is over 21 after an **ace_change**, the sum is assigned a value of "Bust". The sum is encased in a list to show that it has been through the **double** function.

In [6]:
def split(cards):
    return player_hand_advanced(hit([cards[0]])), player_hand_advanced(hit([cards[1]]))

def split_m(cards):
    return player_hand_medium(hit([cards[0]])), player_hand_medium(hit([cards[1]]))

There are two **split** functions, one for the advanced player strategy, and one for the medium player strategy. These functions take as input a hand, and then use each card of the original hand to create a new hand. These two new hands are then run through the appropriate strategy.

In [7]:
def dealer_hand(dealers_cards):
    sdc = sum(dealers_cards)
    if sdc == 21 and len(dealers_cards) == 2:
        return "BJ"
    elif sdc > 21:
        acd = ace_change(dealers_cards)
        if sum(acd) <= 21:
            return dealer_hand(acd)
        else:
            return "Bust"
    elif sdc < 17:
        return dealer_hand(hit(dealers_cards))
    else:
        return sdc

The **dealer_hand** function takes as input a hand of cards, processes this hand according to the usual dealer rules, and then returns the sum of the cards.

In [8]:
def player_hand_advanced(players_cards):
    spc = sum(players_cards)
    if spc == 21 and len(players_cards) == 2:
        spc = "BJ"
        return compare(spc, d_hand, bet)
    elif spc > 21:
        acp = ace_change(players_cards)
        if players_cards == [11,11]:
            return split(players_cards)
        elif sum(acp) <= 21:
            players_cards = acp
            return player_hand_advanced(players_cards)  
        else:
            spc = "Bust"
            return compare(spc, d_hand, bet)
    elif spc >= 17:
        if spc >= 19:
            return compare(spc, d_hand, bet)
        elif 11 not in players_cards:
            if players_cards == [9,9] and (dealer_cards[0] not in [7,10,11]):
                return split(players_cards)
            else:
                return compare(spc, d_hand, bet)
        else:
            if spc == 18:
                if dealer_cards[0] in [2,7,8]:
                    return compare(spc, d_hand, bet)
                elif dealer_cards[0] in [3,4,5,6]:
                    return double(players_cards)
                else:
                    return player_hand_advanced(hit(players_cards))
            else:
                if dealer_cards[0] in [3,4,5,6]:
                    return double(players_cards)
                else:
                    return player_hand_advanced(hit(players_cards))
    elif spc >= 13:
        if players_cards == [7,7]:
            if dealer_cards[0] in [8,9,10,11]:
                return player_hand_advanced(hit(players_cards))
            else:
                return split(players_cards)
        if players_cards == [8,8]:
            return split(players_cards)
        elif 11 not in players_cards:
            if dealer_cards[0] in [2,3,4,5,6]:
                return compare(spc, d_hand, bet)
            else:
                return player_hand_advanced(hit(players_cards))
        else:
            if spc >= 15:
                if dealer_cards[0] in [4,5,6]:
                    return double(players_cards)
                else:
                    return player_hand_advanced(hit(players_cards))
            else:
                if dealer_cards[0] in [5,6]:
                    return double(players_cards)
                else:
                    return player_hand_advanced(hit(players_cards))
    elif spc == 12:
        if players_cards != [6,6]:
            if dealer_cards[0] in [4,5,6]:
                return compare(spc, d_hand, bet)
            else:
                return player_hand_advanced(hit(players_cards))
        else:
            if dealer_cards[0] in [2,3,4,5,6]:
                return split(players_cards)
            else:
                return player_hand_advanced(hit(players_cards))
    elif spc >= 9:
        if spc == 11:
            if dealer_cards[0] == 11:
                return player_hand_advanced(hit(players_cards))
            else:
                return double(players_cards)
        elif spc == 10:
            if dealer_cards[0] in [10,11]:
                return player_hand_advanced(hit(players_cards))
            else:
                return double(players_cards)
        else:
            if dealer_cards[0] in [3,4,5,6]:
                return double(players_cards)
            else:
                return player_hand_advanced(hit(players_cards)) 
    elif players_cards[0] == players_cards[1] and len(players_cards) == 2:
        if players_cards == [4,4]:
            if dealer_cards[0] in [5,6]:
                return split(players_cards)
            else:
                return player_hand_advanced(hit(players_cards))
        else:
            if dealer_cards[0] in [8,9,10,11]:
                return player_hand_advanced(hit(players_cards))
            else:
                return split(players_cards)
    else:
        return player_hand_advanced(hit(players_cards))
                

The **player_hand_advanced** function takes as input a hand of cards and processes it according to the Advanced Blackjack Strategy Table at the following URL: http://www.top5onlinecasino.ca/casino-games/blackjack/strategy/. It eventually sends the cards to the **compare** function described above. This strategy utilizes hits, stays, doubles, and splits; these choices are based on the cards the player currently holds and the visble card of the dealer.

In [9]:
def list_breaker(z):
    if type(z) == int or type(z) == float:
        cl.append(z)
    else:
        for j in z:
            if isinstance(j, list) or isinstance(j, tuple):
                list_breaker(j)
            else:
                cl.append(j)

The **list_breaker** function is used with the advanced and medium strategies (it helps with the **split** functions). It removes numbers from lists/tuples so that the numbers may be individually appended to another list.

In [10]:
def player_hand_medium(players_cards):
    spc = sum(players_cards)
    if spc == 21 and len(players_cards) == 2:
        spc = "BJ"
        return compare(spc, d_hand, bet)
    elif spc > 21:
        acp = ace_change(players_cards)
        if players_cards == [11,11]:
            return split_m(players_cards)
        elif sum(acp) <= 21:
            players_cards = acp
            return player_hand_medium(players_cards)  
        else:
            spc = "Bust"
            return compare(spc, d_hand, bet)
    elif spc >= 17:
        if spc >= 19:
            return compare(spc, d_hand, bet)
        elif 11 not in players_cards:
            if players_cards == [9,9] and (dealer_cards[0] not in [7,10,11]):
                return split_m(players_cards)
            else:
                return compare(spc, d_hand, bet)
        else:
            if spc == 18:
                if dealer_cards[0] in [2,7,8]:
                    return compare(spc, d_hand, bet)
                else:
                    return player_hand_medium(hit(players_cards))
            else:
                return player_hand_medium(hit(players_cards))
    elif spc >= 13:
        if players_cards == [7,7]:
            if dealer_cards[0] in [8,9,10,11]:
                return player_hand_medium(hit(players_cards))
            else:
                return split_m(players_cards)
        if players_cards == [8,8]:
            return split_m(players_cards)
        elif 11 not in players_cards:
            if dealer_cards[0] in [2,3,4,5,6]:
                return compare(spc, d_hand, bet)
            else:
                return player_hand_medium(hit(players_cards))
        else:
            return player_hand_medium(hit(players_cards))
    elif spc == 12:
        if players_cards != [6,6]:
            if dealer_cards[0] in [4,5,6]:
                return compare(spc, d_hand, bet)
            else:
                return player_hand_medium(hit(players_cards))
        else:
            if dealer_cards[0] in [2,3,4,5,6]:
                return split_m(players_cards)
            else:
                return player_hand_medium(hit(players_cards))
    elif players_cards[0] == players_cards[1] and len(players_cards) == 2:
        if players_cards == [4,4]:
            if dealer_cards[0] in [5,6]:
                return split_m(players_cards)
            else:
                return player_hand_medium(hit(players_cards))
        else:
            if dealer_cards[0] in [8,9,10,11]:
                return player_hand_medium(hit(players_cards))
            else:
                return split_m(players_cards)
    else:
        return player_hand_medium(hit(players_cards))

The **player_hand_medium** function does the same thing as the **player_hand_advanced** function, except there are no doubles. Each double has been replaced with a hit.

In [11]:
def player_hand_basic(players_cards):
    spc = sum(players_cards)
    if spc == 21 and len(players_cards) == 2:
        spc = "BJ"
        return compare(spc, d_hand, bet)
    elif spc > 21:
        acp = ace_change(players_cards)
        if sum(acp) <= 21:
            return player_hand_basic(acp)
        else:
            spc = "Bust"
            return compare(spc, d_hand, bet)
    elif spc < 17:
        return player_hand_basic(hit(players_cards))
    else:
        return compare(spc, d_hand, bet)

The **player_hand_basic** function uses the same strategy as the **dealer_hand function**.

------------------------------------------------------

Below, I try out each of the strategies. For each strategy, I use a for loop with 100000 iterations; each iteration is generally equivalent to one hand for the dealer and one hand for the player, although sometimes a player is granted an additonal hand(s) in a given iteration through splitting. Within each iteration, I create a new deck of cards and randomly sample as necessary, each time without setting a seed. This makes each iteration independent of the others.

I use a starter bet = 10 for each hand, but this bet can change depending on what happens in that hand (e.g. a double increases the bet to 20). The result of each hand (i.e. the amount of money the player won) is appended to a list, and after the 100000 iterations are complete, the sum of the list is calculated. This shows, for each strategy, how much money the player would expect to make after 100000 iterations. I then calculate the expected value of each strategy by dividing the sum by the number of hands. Each output below is the sum followed by the expected value.

In [12]:
#Player_hand_advanced

cl = []
reps = 100000
for i in range(reps):
    l = []
    for i in range(2,15):
        l.extend([i,i,i,i])
    l_new = [10 if x >=11 and x <14 else 11 if x == 14 else x for x in l]

    start_cards = random.sample(l_new, 4)
    your_cards = start_cards[0:2]
    dealer_cards = start_cards[2:]
    for i in start_cards:
        l_new.remove(i)
    
    d_hand = dealer_hand(dealer_cards)
    bet = 10
    pha = player_hand_advanced(your_cards)
    
    list_breaker(pha)

s = sum(cl)
ev = s / len(cl)
print(s)
print(ev)

45885.0
0.45111340510249226


In [13]:
#Player_hand_medium

cl = []
reps = 100000
for i in range(reps):
    l = []
    for i in range(2,15):
        l.extend([i,i,i,i])
    l_new = [10 if x >=11 and x <14 else 11 if x == 14 else x for x in l]

    start_cards = random.sample(l_new, 4)
    your_cards = start_cards[0:2]
    dealer_cards = start_cards[2:]
    for i in start_cards:
        l_new.remove(i)
    
    d_hand = dealer_hand(dealer_cards)
    bet = 10
    phm = player_hand_medium(your_cards)
    
    list_breaker(phm)

s = sum(cl)
ev = s / len(cl)
print(s)
print(ev)

-23290.0
-0.22844755711188927


In [14]:
#Player_hand_basic

cl = []
reps = 100000
for i in range(reps):
    l = []
    for i in range(2,15):
        l.extend([i,i,i,i])
    l_new = [10 if x >=11 and x <14 else 11 if x == 14 else x for x in l]

    start_cards = random.sample(l_new, 4)
    your_cards = start_cards[0:2]
    dealer_cards = start_cards[2:]
    for i in start_cards:
        l_new.remove(i)
    
    d_hand = dealer_hand(dealer_cards)
    bet = 10
    phb = player_hand_basic(your_cards)
    
    cl.append(phb)

s = sum(cl)
ev = s / len(cl)
print(s)
print(ev)

-63555.0
-0.63555


As expected, the advanced strategy is the best, followed by the middle strategy, with the basic strategy bringing up the rear. The advanced strategy is the only one out of the three that nets positive returns in the long run.

### Conclusions

My findings were mostly what I expected, i.e. that the advanced strategy would net the best long term result for the player. What I did not necessarily expect is that the advanced strategy would net a long term result greater than zero. This may be due to my assumptions, which are not standard across casinos. For example, many casinos these days use multiple decks instead of one, and they may also use a different shuffling methodology. In a future project, I could explore these potentialities along with the player's ability to count cards.